In [ ]:
import numpy as np
import pandas as pd
from maxvolpy.maxvol import rect_maxvol, maxvol
import random
from itertools import chain, cycle, islice, product
import numba
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import lru_cache
%matplotlib inline

## Does it drop redundant features?

$ 2 $ classes. We are generating a sample. Randomly choose one, then generate features for it. Different classes have different distributions of features. We generate $ n $ samples this way, suppose they all have $ k $ features. Replicate each column $ r $ times, so that we have $ n $ samples each with $ rk $ columns now. Add noise to each column.

Invoke maxvol to select $ k $ columns. See how many duplicates it gets. The hypothesis we're checking is it selects very few duplicates.

In [ ]:
def build_dataset(
    n, class_1_prob, k, min_feature_std, max_feature_std, how_to_duplicate,
    num_duplicates, noise_level, random_seed=None
):
    np.random.seed(random_seed)
    random.seed(random_seed)
    
    # let all features of class 1 have mean 1
    # and let all features of class 2 have mean 3
    class_1_features_means = 2 * np.ones(k)
    class_2_features_means = 4 * np.ones(k)
    
    # now let's choose standard deviations for features
    # for each feature its standard deviation will be the same no matter class 1 or class 2
    features_stds = np.linspace(min_feature_std, max_feature_std, num=k)
    
    # determine how many samples will be of each class
    class_1_num_samples = np.random.binomial(n, class_1_prob)
    class_2_num_samples = n - class_1_num_samples
    
    # now let's actually generate the data
    class_1_dataset = features_stds * np.random.randn(class_1_num_samples, k) + class_1_features_means
    class_2_dataset = features_stds * np.random.randn(class_2_num_samples, k) + class_2_features_means
        
    # in the following array first class_1_num_samples rows contain objects of class 1
    # and all rows after those contain objects of class 2
    dataset = np.concatenate((class_1_dataset, class_2_dataset), axis=0)
    
    return (
        duplicate_and_add_noise(
            dataset, how_to_duplicate, num_duplicates,
            noise_level, min_feature_std, max_feature_std
        ),
        class_1_num_samples,
        class_2_num_samples
    )

In [ ]:
def duplicate_and_add_noise(
    dataset, how_to_duplicate, num_duplicates,
    noise_level, min_feature_std, max_feature_std
):
    k = dataset.shape[1]
    dataset = np.tile(dataset, num_duplicates)
    
    noise_std = noise_level * (min_feature_std + max_feature_std) / 2
    dataset += noise_std * np.random.randn(*dataset.shape)
    
    if how_to_duplicate == "same":
        return dataset
    elif how_to_duplicate == "transforms":
        # add random shifts
        shifts = 10 * dataset.mean(axis=0) * (np.random.rand(k*num_duplicates) - 0.5)
        dataset += shifts
        
        transform_functions = list(islice(
            cycle([
                lambda arr: arr,
                np.exp,
                lambda arr: np.sqrt(np.abs(arr)),
                lambda arr: np.power(arr, 2),
                lambda arr: np.power(arr, 3)
            ]),
            num_duplicates
        ))
        for transform, batch in zip(transform_functions, range(num_duplicates)):
            dataset[:, batch*k:(batch+1)*k] = transform(dataset[:, batch*k:(batch+1)*k])
        return dataset
    else:
        raise ValueError("Incorrect `how_to_duplicate` parameter")

In [ ]:
k = 200 # number of true features
num_duplicates = 5
dataset, class_1_num_samples, class_2_num_samples = build_dataset(
    n=10000,
    class_1_prob=0.75,
    k=k,
    min_feature_std=0.5,
    max_feature_std=1.5,
    how_to_duplicate="transforms",
    num_duplicates=num_duplicates,
    noise_level=0.1
)

In [ ]:
print(dataset[:class_1_num_samples, 0].mean())
print(dataset[class_1_num_samples:, 0].mean())

# difference should be approximately 2

In [ ]:
print(dataset[:class_1_num_samples, k].mean())
print(dataset[class_1_num_samples:, k].mean())

print(dataset[:class_1_num_samples, k].std())
print(dataset[class_1_num_samples:, k].std())

We see that for each $ i \in \{ 0, \dots, \text{ num_true_features} \} $ correlation of each column with number $ i + j \text{ num_true_features} $ is far from zero.

In [ ]:
i = 1
pd.DataFrame(dataset[:, range(i, i + (num_duplicates-1)*k + 1, k)]).corr()

Now we want to choose the best k-column submatrix using `rect_maxvol`.

First let's try simply choosing k random rows.

In [ ]:
normalized_dataset = (dataset - dataset.mean(axis=0)) / dataset.std(axis=0)

In [ ]:
normalized_dataset.mean(axis=0)
# all values should be close to 0

In [ ]:
normalized_dataset.std(axis=0)
# all values should be close to 1

In [ ]:
def choose_features(dataset, k, samples_choice):
    n = dataset.shape[0]
    if samples_choice == "random":
        samples_subset_indices = np.random.choice(n, size=k, replace=False)
    elif samples_choice == "rect_maxvol":
        raise ValueError("Not implemented yet")
    else:
        raise ValueError("Incorrect samples_choice parameter")
    features_subset_indices = rect_maxvol(dataset[samples_subset_indices, :].T, minK=k, maxK=k, tol=0.05)[0]
    return features_subset_indices

In [ ]:
def calculate_percentage_uniq_features(features_subset_indices, k):
    return len(np.unique(features_subset_indices % k)) / len(features_subset_indices)

In [ ]:
chosen_features_indices = choose_features(normalized_dataset, k=k, samples_choice="random")
#print(chosen_features_indices)
#print(chosen_features_indices % k)
print(calculate_percentage_uniq_features(chosen_features_indices, k)) # more is better

Let's also calculate expected percentage of uniq features if we chose columns randomly.

We have $ k $ true features, each replicated $ r $ times.

Math is difficult, so I'll calculate this by experiments.

In [ ]:
@lru_cache(maxsize=1000)
@numba.jit(nopython=True)
def calculate_expected_uniq_features(k, num_duplicates):
    total_count = 0
    num_experiments = 10000
    for i in range(num_experiments):
        chosen_true_features = np.random.choice(k*num_duplicates, k, replace=False) % k
        num_distinct_features = len(np.unique(chosen_true_features))
        total_count += num_distinct_features
    return total_count / (k * num_experiments)

In [ ]:
calculate_expected_uniq_features(k, num_duplicates)

# our hypothesis is that this number will be significantly lower than the number when
# we choose features using maxvol

## Plot a plot

In [ ]:
def do_experiments(duplication_method, k_values, min_feature_stds, num_duplicates, class_1_prob=0.75):
    assert len(k_values) == len(min_feature_stds)
    for (k, min_feature_std) in tqdm(list(zip(k_values, min_feature_stds))):
        num_experiments = 10
        for i in range(num_experiments):
            dataset, class_1_num_samples, class_2_num_samples = build_dataset(
                n=10000,
                class_1_prob=class_1_prob,
                k=k,
                min_feature_std=min_feature_std,
                max_feature_std=min_feature_std+1,
                how_to_duplicate=duplication_method,
                num_duplicates=num_duplicates,
                noise_level=0.1,
            )
            normalized_dataset = (dataset - dataset.mean(axis=0)) / dataset.std(axis=0)
            chosen_features_indices = choose_features(normalized_dataset, k=k, samples_choice="random")
            experiments_data.append({
                "k": k,
                "std": min_feature_std+0.5,
                "duplication_method": duplication_method,
                "maxvol feature selection D(I)": calculate_percentage_uniq_features(chosen_features_indices, k),
                "random feature selection D(I)": calculate_expected_uniq_features(k, num_duplicates),
                "class_1_prob": class_1_prob
            })

In [ ]:
experiments_data = []
k=60

In [ ]:
do_experiments("same", [k]*10, min_feature_stds=np.linspace(0.1, 6, num=10), num_duplicates=num_duplicates)

In [ ]:
do_experiments("transforms", [k]*10, min_feature_stds=np.linspace(0.1, 6, num=10), num_duplicates=num_duplicates)

In [ ]:
k_values = [int(4 * 100**(i/19)) for i in range(17)]
# Now do experiment for changing k
do_experiments("same", k_values, min_feature_stds=[3] * 17, num_duplicates=num_duplicates)

In [ ]:
do_experiments("transforms", k_values, min_feature_stds=[3] * 17, num_duplicates=num_duplicates)

In [ ]:
df = pd.DataFrame(experiments_data)

In [ ]:
df.sample(5)

In [ ]:
df

In [ ]:
gp_same = df[(df.duplication_method == "same") & (df.k == k) & (df["class_1_prob"] == 0.75)] \
    [["std", "maxvol feature selection D(I)", "random feature selection D(I)"]] \
    \
    .groupby("std")
fig, ax = plt.subplots()
gp_same.mean().plot(yerr=2*gp_same.std(), ax=ax)
ax.set_ylim(-0.05, 1.05)
ax.set_ylabel("percentage of distinct features D(I)")
ax.set_xlabel('standard deviation of "true features", $ (\sigma_{min} + \sigma_{max})/2 $', usetex=True)

In [ ]:
gp_transforms = df[(df.duplication_method == "transforms") & (df.k == k) & (df["class_1_prob"] == 0.75)] \
    [["std", "maxvol feature selection D(I)", "random feature selection D(I)"]] \
    .groupby("std")
fig, ax = plt.subplots()
gp_transforms.mean().plot(yerr=2*gp_same.std(), ax=ax)
ax.set_ylim(-0.05, 1.05)
ax.set_ylabel("percentage of distinct features D(I)")
ax.set_xlabel('standard deviation of "true features", $ (\sigma_{min} + \sigma_{max})/2 $', usetex=True)

In [ ]:
# plot for changing k
gp_k_same = df[(df.duplication_method == "same") & (df["std"] == 3.5) & (df["class_1_prob"] == 0.75)] \
    [["k", "maxvol feature selection D(I)", "random feature selection D(I)"]] \
    .groupby("k")
fig, ax = plt.subplots()
gp_k_same.mean().plot(yerr=2*gp_k_same.std(), ax=ax)
ax.set_ylim(-0.05, 1.05)
ax.set_ylabel("percentage of distinct features, D(I)")
ax.set_xlabel("number of true features, k")


In [ ]:
# plot for changing k
gp_k_transforms = df[(df.duplication_method == "transforms") & (df["std"] == 3.5) & (df["class_1_prob"] == 0.75)] \
    [["k", "maxvol feature selection D(I)", "random feature selection D(I)"]] \
    .groupby("k")
fig, ax = plt.subplots()
gp_k_transforms.mean().plot(yerr=2*gp_k_same.std(), ax=ax)
ax.set_ylim(-0.05, 1.05)
ax.set_ylabel("percentage of distinct features, D(I)")
ax.set_xlabel("number of true features, k")


The first two plots seem weird to me, so let's check what will happen if we only have one class

In [ ]:
do_experiments("same", [k]*10, min_feature_stds=np.linspace(0.1, 6, num=10), num_duplicates=num_duplicates, class_1_prob=1)

In [ ]:
do_experiments("transforms", [k]*10, min_feature_stds=np.linspace(0.1, 6, num=10), num_duplicates=num_duplicates, class_1_prob=1)

In [ ]:
df = pd.DataFrame(experiments_data)

In [ ]:
gp_same = df[(df.duplication_method == "same") & (df["class_1_prob"] == 1)][["std", "maxvol feature selection D(I)", "random feature selection D(I)"]] \
    .groupby("std")
fig, ax = plt.subplots()
gp_same.mean().plot(yerr=2*gp_same.std(), ax=ax)
ax.set_ylim(-0.05, 1.05)
ax.set_ylabel("percentage of distinct features, D(I)")
ax.set_xlabel('standard deviation of "true features", $ (\sigma_{min} + \sigma_{max})/2 $', usetex=True)

In [ ]:
gp_same = df[(df.duplication_method == "transforms") & (df["class_1_prob"] == 1)][["std", "maxvol feature selection D(I)", "random feature selection D(I)"]] \
    .groupby("std")
fig, ax = plt.subplots()
gp_same.mean().plot(yerr=2*gp_same.std(), ax=ax)
ax.set_ylim(-0.05, 1.05)
ax.set_ylabel("percentage of distinct features, D(I)")
ax.set_xlabel('standard deviation of "true features", $ (\sigma_{min} + \sigma_{max})/2 $', usetex=True)